# Исследование надёжности заёмщиков


**Задача** от кредитного отдел банка: разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок, чтобы учитывать это при построении модели кредитного скоринга (системы, оценивающей способность потенциального заёмщика вернуть кредит банку).

## Общая информация

In [1]:
import pandas as pd                       # импорт библиотеку pandas

from pymystem3 import Mystem              # Импортировали библиотеки для лемматизации
from collections import Counter

data = pd.read_csv('data.csv')            # сохранение данных из файла в переменной data
data.info()                               # методом info() оцениваем объем данных в таблице
data.head(15)                             # просматриваем первые 15 строк


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

Итак, в таблице 12 столбцов. 

Согласно документации к данным:
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

Тип данных: 
float64(2 - доход и стаж), 
int64(5 - количество детей, возраст, идентификаторы образования и семейного статуса и наличие задолженности по кредитам), object(5 - образование, семейный статус, пол, тип занятости, цель получения кредита).

Так как количество значений в столбцах различается, то в данных есть пропущенные значения (столбцы 'days_employed', 'total_income').

При первом просмотре таблицы данных выявляются следующие проблемы:

- трудовой стаж (значения в столбце 'days_employed') может принимать как положительные, так и отрицательные значения, отдельные значения слишком велики и не могут быть днями стажа,
- строчные буквы сочетаются с прописными,
- данные по стажу не требуют формата float и могут быть заменены на целочисленные int,
- цели требуют лемматизации.



##  Предобработка данных

### Обработка пропусков

Для начала приведем значения столбца стажа к корректным. Узнаем, какие значения стажа считаются не в днях, а в часах. Для этого найдем максимальный возраст, вычтем из него первые 14 лет детства и умножим на 365 дней. Это будет предельным значением, после которого значение ячейки со стажем будем делить на 24 часа.

In [2]:
max_days_employed = (data['dob_years'].max() - 14) * 365
max_days_employed

22265

Теперь заменим все значения на модули (уберем отрицательные), а значения, превышающие максимально возможный стаж в днях, поделим на 24.

In [3]:
data['days_employed'] = data['days_employed'].apply(abs)   #  модули (убрали отрицательные)

data.loc[data['days_employed'] > max_days_employed, 'days_employed'] = (
    data.loc[data['days_employed'] > max_days_employed, 'days_employed'] / 24
)

data.loc[4, 'days_employed']                               # проверяем на примере строки 4

14177.753001950914

Заменяем пропуски в стаже на медианные значения.

In [4]:
print('Пропусков до:', data['days_employed'].isna().sum())

for age in range(data['dob_years'].min(), data['dob_years'].max()): # перебираем все возможные значения возраста
    
    mediana = data.loc[data['dob_years']==age, 'days_employed'].median() # находим медианное значение стажа для этого возраста
    data.loc[(data['days_employed'].isna()) & (data['dob_years']==age), 'days_employed'] = mediana 
                                                             # значение NaN заменяем на медианное значение для такого возраста

print('Пропусков после:', data['days_employed'].isna().sum()) 

Пропусков до: 2174
Пропусков после: 0


Теперь заменяем пропуски в столбце дохода на медианные значения.

In [5]:
print('Пропусков до:', data['total_income'].isna().sum())

for age in range(data['dob_years'].min(), data['dob_years'].max()): # перебираем все возможные значения возраста
    
    mediana = data.loc[data['dob_years']==age, 'total_income'].median() # находим медианное значение дохода для этого возраста
    data.loc[(data['total_income'].isna()) & (data['dob_years']==age), 'total_income'] = mediana 
                                                             # значение NaN заменяем на медианное значение для такого возраста

print('Пропусков после:', data['total_income'].isna().sum()) 


display(data.loc[12, ['days_employed', 'total_income']])     # проверяем на примере строки 12

Пропусков до: 2174
Пропусков после: 0


days_employed    14902.9
total_income      122934
Name: 12, dtype: object

Убираем прописные буквы в столбце 'education' и затем проверяем корректность значений.

In [6]:
data['education'] = data['education'].str.lower()
print(data['education'].unique())

['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']


Проверяем корректность данных возраста заемщиков (количество нулевых,отрицательных значений и возраста до 18 лет). Выведем на экран первые значения столбца 'dob_years' в порядке возрастания возраста и количество таких значений.

In [7]:
print(data.groupby('dob_years')['dob_years'].count().head()) 

dob_years
0     101
19     14
20     51
21    111
22    183
Name: dob_years, dtype: int64


Считаем значение 0 в возрасте пропуском и заменяем на медианные значения, в остальном всё выглядит корректно.

In [8]:
mediana = int(data['dob_years'].median())                                 # находим медианное значение возраста, переводим в int
data.loc[data['dob_years']==0, 'dob_years'] = mediana 

print('Осталось нулей:', data[data['dob_years']==0]['dob_years'].count()) # Проверяем, остались ли нули   

Осталось нулей: 0


Находим уникальные значения оставшихся столбцов, чтобы исключить в них ошибки. Используем цикл.

In [9]:
column_dict == ['education_id', 'family_status', 'family_status_id', 'gender', 'income_type', 'debt', 'children']
for col in column_dict: 
    print(data[col].unique())

[0 1 2 3 4]
['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']
[0 1 2 3 4]
['F' 'M' 'XNA']
['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']
[0 1]
[ 1  0  3  2 -1  4 20  5]


В столбце 'gender' есть неизвестное значение 'XNA'. Уточним данные.

In [11]:
print(data['gender'].value_counts())

F      14236
M       7288
XNA        1
Name: gender, dtype: int64


Заменим на наиболее часто встречающееся значение этого столбца.

In [12]:
data.loc[data['gender']=='XNA', 'gender'] = 'F'

В столбце 'children' есть значение "-1". Скорее всего, оно означает отсутствие детей, заменим такие на "0". Значение "-20" похоже на опечатку, заменим на "2".

In [13]:
data.loc[data['children']== -1, 'children'] = 0
data.loc[data['children']== 20, 'children'] = 2

print(data['children'].unique())  # Проверяем.

[1 0 3 2 4 5]


In [14]:
data.info() # Выводим на экран информацию по пролучившейся таблице.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

Были обработаны имевшиеся в таблице пропуски путем заменены на медианные значения. Некорректные значения дней стажа заменены на  корректные с помощью функции abs и деления на 24. Пропуски в возрасте, обозначенные как 0, заменены на медианные. Убраны прописные буквы в столбце 'education'. Также исправлены значение 'XNA' в столбце 'gender' на 'F' и количество детей с -1 на 0 и с 20 на 2.
Получившаяся таблица не имеет пропущенных значений, тип данных не изменился.

### Замена типа данных

Cледует отметить, что имеет смысл заменить тип количества дней стажа 'days_employed' на int. Это нагляднее, соответствует общепринятому подсчету рабочих дней / стажа в целых числах.  Заменяем float на int.

In [15]:
data['days_employed'] = data['days_employed'].astype('int')  # Применяем функцию int с помощью метода apply к указанному столбцу

print(data['days_employed'].head())  # Проверяем

0     8437
1     4024
2     5623
3     4124
4    14177
Name: days_employed, dtype: int64


**Вывод**

Изменили стаж 'days_employed' на int. 
Можно поступить подобным образом и со средним заработком, но тип float не дает преимущества в данном конкретном случае, а доход может считаться до копейки и даже долей копейки, поэтому этот столбец оставим без изменений.

### Обработка дубликатов

Для дальнейшей корректной работы с таблицей необходимо удалить дубликаты. Применяем метод duplicated() с параметром keep = False - вывод всех совпадений и сортируем по всем столбцам. 

In [16]:
print('Всего дубликатов:', data.duplicated().sum())             # Найдем их общее количество.

data[data.duplicated(keep = False)].sort_values(by = list(data.columns))

Всего дубликатов: 71


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
15892,0,690,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,123162.782874,сделка с подержанным автомобилем
19321,0,690,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,123162.782874,сделка с подержанным автомобилем
3452,0,1315,29,высшее,0,женат / замужем,0,M,сотрудник,0,146617.644239,покупка жилой недвижимости
18328,0,1315,29,высшее,0,женат / замужем,0,M,сотрудник,0,146617.644239,покупка жилой недвижимости
4216,0,1420,30,среднее,1,женат / замужем,0,M,сотрудник,0,146944.669481,строительство жилой недвижимости
...,...,...,...,...,...,...,...,...,...,...,...,...
9238,2,1615,34,среднее,1,женат / замужем,0,F,сотрудник,0,155358.963793,покупка жилья для сдачи
9013,2,1799,36,высшее,0,женат / замужем,0,F,госслужащий,0,158681.482178,получение образования
14432,2,1799,36,высшее,0,женат / замужем,0,F,госслужащий,0,158681.482178,получение образования
11033,2,1891,39,среднее,1,гражданский брак,1,F,сотрудник,0,149429.152630,сыграть свадьбу


Количество дубликатов по сравнению с общим числом строк не велико. Возможная причина появления дубликатов - замена пропущенных значений медианными. Поскольку таблица не имеет столбца с идентификатором заемщика, истинность дубликата полностью проверить не представляется возможным.
Удаляем дубликаты с помощью метода drop_duplicates(), переименовав строки (без создания нового столбца индекса).

In [17]:
data = data.drop_duplicates().reset_index(drop=True) 
data.info()               # проверяем таблицу на предмет пропущенных значений

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null float64
purpose             21454 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


**Вывод**

Количество строк в исходной таблице  21525, количество строк-дубликатов 71. После удаления дубликатов число строк соответственно уменьшилось и составило 21454. Тип данных не изменился.

### Лемматизация

Осталось добавить столбец с лемматизированными целями для дальнейшего анализа заемщиков. Лемматизируем, предварительно создав единую строку из всех уникальных значений столбца 'purpose'.

In [18]:
m = Mystem()                                              # Создаем объект-лемматизатор и присваиваем переменной m это значение 
  
lemmas = m.lemmatize(' '.join(data['purpose'].unique()))  # Лемматизируем

Выведем полученные леммы и частоту их повторений.

In [19]:
Counter(lemmas)  

Counter({'покупка': 10,
         ' ': 96,
         'жилье': 7,
         'приобретение': 1,
         'автомобиль': 9,
         'дополнительный': 2,
         'образование': 9,
         'сыграть': 1,
         'свадьба': 3,
         'операция': 4,
         'с': 5,
         'на': 4,
         'проведение': 1,
         'для': 2,
         'семья': 1,
         'недвижимость': 10,
         'коммерческий': 2,
         'жилой': 2,
         'строительство': 3,
         'собственный': 1,
         'подержать': 2,
         'свой': 4,
         'со': 1,
         'заниматься': 2,
         'сделка': 2,
         'получение': 3,
         'высокий': 3,
         'профильный': 1,
         'сдача': 1,
         'ремонт': 1,
         '\n': 1})

Как видим, можно выделить наиболее часто встречающиеся категории: с покупкой жилья и недвижимости, автомобилями, свадьбой, образованием. Остальное можно отнести к категории "прочее".

Объявим функцию, возвращающую категорию целии кредита. Применим ее для создания нового столбца "Категория", применив к столбцу 'purpose' объявленную функцию. Выведем количество значений в каждой категории.

In [20]:
def categories(category_value):                                # Объявили функцию
    category_lemmas = m.lemmatize(category_value)
    if 'образование' in category_lemmas:
        return 'образование'
    elif 'автомобиль' in category_lemmas:
        return 'автомобиль'
    elif 'свадьба' in category_lemmas:
        return 'свадьба'
    elif 'операция' in category_lemmas:
        return 'прочее'
    elif ('жилье' in category_lemmas) or ('недвижимость' in category_lemmas):
        return 'недвижимость'
    else:
        return 'прочее'
 
data['category_purpose'] = data['purpose'].apply(categories)   # Создадим новый столбец 'category'

display(data.loc[:, ['purpose', 'category_purpose']].head(15)) # Проверяем на первых 15 строках получившиеся даннные

print('Итого по категориям:\n', data['category_purpose'].value_counts())  # Подсчитываем количество значений в каждой категории
print('\n Пропущенных значений:', data['category_purpose'].isna().sum())

,purpose,category_purpose
0,покупка жилья,недвижимость
1,приобретение автомобиля,автомобиль
2,покупка жилья,недвижимость
3,дополнительное образование,образование
4,сыграть свадьбу,свадьба
5,покупка жилья,недвижимость
6,операции с жильем,прочее
7,образование,образование
8,на проведение свадьбы,свадьба
9,покупка жилья для семьи,недвижимость


Итого по категориям:
 недвижимость    8207
автомобиль      4306
образование     4013
прочее          2604
свадьба         2324
Name: category_purpose, dtype: int64

 Пропущенных значений: 0


**Вывод**

Имеющиеся цели кредита свели к пяти наиболее часто встречающимся категориям: недвижимость, автомобиль, образование, прочее, свадьба. 

### Категоризация данных

Разберемся, какие еще категории, представленные в таблице, могут быть полезны для дальнейшего исследования. Для этого для имеющихся категориальных переменных построим фреймы, содержащие их уникальные значения.

Первое - семейное положение (в качестве индекса использовали столбец 'family_status_id').

In [21]:
data[['family_status_id', 'family_status']].drop_duplicates().set_index('family_status_id') 

,family_status
family_status_id,
0,женат / замужем
1,гражданский брак
2,вдовец / вдова
3,в разводе
4,Не женат / не замужем


Количество детей просчитаем методом value_counts().

In [22]:
data['children'].value_counts()

0    14138
1     4808
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

Количественную переменную total_income (доход) приведем к категориальным значениям. Для этого сначала исследуем ее статистические значения.

In [23]:
min_revenue = data['total_income'].min()
print(f'Минимальный доход: {min_revenue:.2f} руб.')

max_revenue = data['total_income'].max()
print(f'Максимальный доход: {max_revenue:.2f} руб.')

median_revenue = data['total_income'].median()
print(f'Медианное значение дохода: {median_revenue:.2f} руб.')

mean_revenue = data['total_income'].mean()
print(f'Среднее значение дохода: {mean_revenue:.2f} руб.')

Минимальный доход: 20667.26 руб.
Максимальный доход: 2265604.03 руб.
Медианное значение дохода: 146499.74 руб.
Среднее значение дохода: 165270.98 руб.


Имеем огромный разброс в доходах от 20 тысяч до 2,26 млн рублей. 
Медианное и среднее значения можно считать близкими друг к другу, это означает, что за средний доход заемщиков можно принять значения в пределах 140-170 тысяч рублей, плюс-минус еще 20 тысяч.
Переведем количественную переменную дохода в качественную, определив границы следующим образом:
- низкий (до 70 000) 
- ниже среднего (больше 70 000 до 120 000)
- средний (больше 120 000 до 190 000)
- выше среднего (больше 190 000 до 400 000)
- высокий (более 400 000)

Определим функцию, позволяющую определить категорию дохода. Создадим новый столбец 'total_income_id', применив к столбцу 'total_income' объявленную функцию.

In [24]:
def revenue_category(revenue):  # Определяем функцию
    if revenue <= 70000:
        return 'низкий'
    elif revenue <= 120000:
        return 'ниже среднего'
    elif revenue <= 190000:
        return 'средний'
    elif revenue <= 400000:
        return 'выше среднего'
    else:
        return 'высокий'

 
data['total_income_id'] = data['total_income'].apply(revenue_category)  # Создали новый столбец 
    
display(data.loc[:, ['total_income', 'total_income_id']].head())        # Проверяем на первых строках получившиеся даннные

print('Итого по категориям:\n', data['total_income_id'].value_counts()) # Подсчитываем количество значений в каждой категории

,total_income,total_income_id
0,253875.639453,выше среднего
1,112080.014102,ниже среднего
2,145885.952297,средний
3,267628.550329,выше среднего
4,158616.077870,средний


Итого по категориям:
 средний          8798
ниже среднего    5432
выше среднего    5222
низкий           1474
высокий           528
Name: total_income_id, dtype: int64


**Вывод:**
- Имеем пять категорий семейного статуса. 
- Количество детей также можно отнести к категориальной переменной, значения от 0 (нет детей) до пяти.
- Получившиеся данные по категориям дохода можно считать действительными, так как распределение по категориям похоже на близкое к истине. Основная часть попала в три средние категории, причем в "средней" наибольшее количество. Значительно меньше в категории "низкий" доход и еще меньше в категории "высокий".

##  Несколько вопросов

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

Для ответа на этот вопрос построим сводную таблицу методом pivot_table(). Затем переименуем названия получившихся столбцов. 

Заменим возможные пропущенные значения на 0.В дальнейшем при определении процентного соотношения деления на ноль не будет, так как любая категория количества детей встречается в таблице не менее одного раза. В применении try-except необходимости нет.
    
Далее найдем общее количество заемщиков в категории (посчитаем процентное отношение не вернувших кредит к общему числу) и занесем эти значения в новый столбец 'conversion_rate'. После отсортируем по доле невозврата в убывающем порядке.

In [25]:

data_pivot = data.pivot_table(index=['children'], # Группировка по столбцу 'children' - уникальные значения будут строками
                              columns=['debt'],   # Группировка по столбцу 'debt' - его уникальные значения будут столбцами
                              values=['gender'],  # Для подсчета количества можем взять любой столбец, это не принципиально
                              aggfunc=['count'])  # Считаем количество функцией 'count', применив ее к столбцу 'gender'

data_pivot.columns = ['no_debt', 'debt']                      # Названия получившихся столбцов

data_pivot = data_pivot.fillna(0)                             # Заменили возможные пропущенные значения на 0 
    
data_pivot['number_borrowers'] = data_pivot['no_debt'] + data_pivot['debt'] # Нашли общее количество 

data_pivot['percentage_non_return'] = data_pivot['debt'] / data_pivot['number_borrowers']  # Посчитали процентное отношение 
   
display(data_pivot.sort_values(
    by='percentage_non_return', 
    ascending=False)                                          # Отсортируем по доле невозврата  в убывающем порядке
       )  

,no_debt,debt,number_borrowers,percentage_non_return
children,,,,
4,37.0,4.0,41.0,0.097561
2,1926.0,202.0,2128.0,0.094925
1,4364.0,444.0,4808.0,0.092346
3,303.0,27.0,330.0,0.081818
0,13074.0,1064.0,14138.0,0.075258
5,9.0,0.0,9.0,0.000000


Из данных вытекает, что категоризацию по количеству детей делать не верно. На основе этой таблицы по всем имеющимся значениям сделаем вывод про некорректные данные по категориям 3, 4, 5 детей.

То есть, ввиду слишком малого количества заемщиков, имеющих 3, 4, 5 детей, и, как следствие, непрезентативности выборки в этих категориях, а также ввиду близости значений доли невозврата в них к имеющим 1 и 2 детей, можно было бы распределить категории на три: "нет детей", "один ребенок", "два и более детей". Но на итоговый вывод это не повлияло бы, так как, судя по получившимся данным, вообще имеет смысл разбить всего на две категории: "есть дети" и "нет детей".

Подсчитаем общий процент невозврата кредита по всем заемщикам, имеющим детей.


In [26]:
debt_children = data_pivot['debt'].sum() - data_pivot.loc[0, 'debt']              # Сколько заемщиков c детьми имеют долг

number_borrowers_children = data_pivot['number_borrowers'].sum() - data_pivot.loc[0, 'number_borrowers'] # Всего с детьми

percentage_non_return_children = debt_children / number_borrowers_children                            # Находим отношение

print(f'Доля невозврата среди заемщиков, имеющих детей: {percentage_non_return_children:.2%}')    

Доля невозврата среди заемщиков, имеющих детей: 9.25%


Создадим новую таблицу по новым критериям (есть дети - нет детей) и посчитаем конверсию.    

In [27]:
def children_category(children):                                # Определим функцию.
    if children == 0:
        return 'Бездетные' 
    else:
        return 'Есть дети (1 и более)'  


data['children_id'] = data['children'].apply(children_category) # Создали новый столбец 'children_id'

data_pivot = data.pivot_table(index=['children_id'], 
                              columns=['debt'],   
                              values=['gender'],  
                              aggfunc=['count'])  

data_pivot.columns = ['no_debt', 'debt'] 
data_pivot = data_pivot.fillna(0) 
        
data_pivot['number_borrowers'] = data_pivot['no_debt'] + data_pivot['debt'] # Нашли общее количество заемщиков в категории

data_pivot['percentage_non_return'] = (100 * data_pivot['debt'] / data_pivot['number_borrowers']).round(2) 
        #  Посчитали конверсию в процентах и округлили с помощью round() до 2 знаков после запятой
    
display(data_pivot.sort_values(by='percentage_non_return', ascending=False)) 

,no_debt,debt,number_borrowers,percentage_non_return
children_id,,,,
Есть дети (1 и более),6639,677,7316,9.25
Бездетные,13074,1064,14138,7.53


**Вывод**

- Не имеющие детей заемщики чаще возвращают кредит, чем имеющие детей (7,51% против 9,19%). 
- Являться единственным критерием ввиду небольшого разброса результатов не может. 

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

Сдедаем сводную таблицу. Заменим возможные пропущенные значения на 0 аналогично предыдущей сводной таблице. Найдем общее количество заемщиков в категории, посчитаем процентное отношение не вернувших кредит к общему числу - новый столбец 'conversion_rate', и отсортируем по доле невозврата в убывающем порядке.  

In [28]:
data_pivot = data.pivot_table(index=['family_status_id', 'family_status'], 
                              columns=['debt'], 
                              values=['gender'], # Для подсчета количества можем взять любой столбец
                              aggfunc=['count'])

data_pivot.columns = ['no_debt', 'debt']         # Объявили названия получившихся столбцов

data_pivot = data_pivot.fillna(0)                # Заменим возможные пропущенные значения  
        
data_pivot['number_borrowers'] = data_pivot['no_debt'] + data_pivot['debt'] # Нашли общее количество заемщиков в категории

data_pivot['percentage_non_return'] = data_pivot['debt'] / data_pivot['number_borrowers'] 
    # Посчитали процентное отношение не вернувших кредит к общему числу - новый столбец 'conversion_rate'

display(data_pivot.sort_values(by='percentage_non_return', ascending=False))  # Отсортировали в убывающем порядке

,,no_debt,debt,number_borrowers,percentage_non_return
family_status_id,family_status,,,,
4,Не женат / не замужем,2536,274,2810,0.097509
1,гражданский брак,3763,388,4151,0.093471
0,женат / замужем,11408,931,12339,0.075452
3,в разводе,1110,85,1195,0.071130
2,вдовец / вдова,896,63,959,0.065693


**Вывод**

- По полученной сводной таблице можно сделать вывод, что не женатые (не замужние) категории заемщиков имеют самый высокий процент невозврата: 9,75%. Состоящие в гражданском браке находятся близко к предыдущей категории не женатых, у них 9,35%. 
- Те, кто состоит или состоял в браке, практически на 2% от общего числа надежнее, у них процент невозврата от общего числа 7,5% и менее. 
- Так как доля вдовцов, как и находящихся в разводе, всего порядка 4% от всех данных таблицы, делать вывод об их большей надежности не совсем статистически корректно. Возможно, имеет смысл объединить их в одну категорию, так как данные по проценту невозврата близки. 
- Тенденция прослеживается, но не является критичной.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Поступим аналогично.

In [29]:
data_pivot = data.pivot_table(index=['total_income_id'], 
                              columns=['debt'], 
                              values=['gender'],     # Для подсчета количества можем взять любой столбец
                              aggfunc=['count'])

data_pivot.columns = ['no_debt', 'debt']             # Объявили названия получившихся столбцов

data_pivot = data_pivot.fillna(0)                    # Заменим возможные пропущенные значения на 0 
        
data_pivot['number_borrowers'] = data_pivot['no_debt'] + data_pivot['debt'] # Нашли общее количество заемщиков в категории

data_pivot['percentage_non_return'] = data_pivot['debt'] / data_pivot['number_borrowers'] # Посчитали процентное отношение
    

display(data_pivot.sort_values(by='percentage_non_return', ascending=False))

,no_debt,debt,number_borrowers,percentage_non_return
total_income_id,,,,
средний,8031,767,8798,0.087179
ниже среднего,4977,455,5432,0.083763
выше среднего,4835,387,5222,0.074110
низкий,1373,101,1474,0.068521
высокий,497,31,528,0.058712


**Вывод**

- Аккуратнее всего возвращают кредиты люди с самым высоким доходом и с самым низким, а хуже всего с возвратами состоит дело у имеющих средний и ниже среднего доход.
- По сравнению с предыдущими критериями (наличие детей и семейное положение) доход влияет на способность вернуть кредит не столь сильно (худший результат 8,7% против 9,75% - семейное положение).
- В крайних категориях подсчитанная конверсия отличается сильнее, чем по предыдущим критериям, практически на 3%. 
- В долях: количество невозвратов в полтора раза больше среди тех, кто имеет средний доход, чем среди имеющих высокий.
- Прямой зависимости от уровня дохода нет, хотя определенные тенденции имеются.

### Как разные цели кредита влияют на его возврат в срок?

In [30]:
data_pivot = data.pivot_table(index=['category_purpose'], 
                              columns=['debt'], 
                              values=['gender'], 
                              aggfunc=['count'])

data_pivot.columns = ['no_debt', 'debt'] 
data_pivot['number_borrowers'] = data_pivot['no_debt'] + data_pivot['debt'] 
data_pivot['percentage_non_return'] = data_pivot['debt'] / data_pivot['number_borrowers'] # Посчитали столбец 'conversion_rate'

display(data_pivot.sort_values(by='percentage_non_return', ascending=False)) # Отсортировали

,no_debt,debt,number_borrowers,percentage_non_return
category_purpose,,,,
автомобиль,3903,403,4306,0.093590
образование,3643,370,4013,0.092200
свадьба,2138,186,2324,0.080034
прочее,2399,205,2604,0.078725
недвижимость,7630,577,8207,0.070306


**Вывод**

- Хуже всего отдают кредиты, взятые на покупку автомобиля и образование (9 из 100 человек), чуть лучше обстоят дела с возвратом у тех, кто брал кредит на свадьбу и прочие операции (8 из 100), а самые дисциплинированные заемщики те, кто брал кредит на покупку жилья и недвижимости (7 из 100).
- По этому критерию наименьший разброс значений невозврата.
- Зависимость есть, но не критичная.

##  Общий вывод

Предоставленная таблица является репрезентативной. 
В среднем по всем примененным критериям количество заемщиков, не вернувших кредиты, в крайних группах отличаются примерно в полтора раза. 
Нигде нет значений выше 9,75% или ниже 5,9%. 
**Нет ни одного критерия, который мог бы выступать в качестве основного. 
Следовательно, при построении модели кредитного скоринга необходимо рассматривать всю совокупность критериев.**